In [1]:
import json
import pandas as pd


BASE_PATH = "../base_completa/boatos_br_corpus.json"
CATEGORIAS_PROCESSADAS_PATH = "./categorias/categorias_processadas.json"
ORIGENS_PROCESSADAS_PATH = "./origem/origens_processadas.json"

In [2]:
database = pd.read_json(BASE_PATH).to_dict('records')

### Carregar dados processados

In [3]:
def loadJson(path:str):
    aux = []
    with open(path,'r') as f:
        aux = json.load(f)
        f.close()
    return aux

def normalizarDadoProcessado(dado:str,listaNormalizacao:list) -> int:
    return listaNormalizacao[dado] if listaNormalizacao[dado] != None else -1
 

In [4]:
categoriasDict = loadJson(CATEGORIAS_PROCESSADAS_PATH)
origensDict = loadJson(ORIGENS_PROCESSADAS_PATH) 
rotulosDict = {
    'falso': 0,
    'verdade': 1,
    'meia-verdade': 2
}

In [5]:
for data in database:
    data['categorias'] = [normalizarDadoProcessado(x,categoriasDict) for x in data['categorias']]
    data['origem'] = normalizarDadoProcessado(data['origem'],origensDict)
    data['data-publicacao'] = pd.to_datetime(data['data-publicacao']).timestamp()
    data['rotulo'] = normalizarDadoProcessado(data['rotulo'],rotulosDict)

In [6]:
with open(f"../base_processada/boatos_br_corpus_processada_att.json",'w',encoding="utf8") as f:
    json.dump(database,f,indent=3,allow_nan=True,ensure_ascii = False)
    f.close()